In [17]:
import pandas as pd

import numpy as np
from utils import config, utils

In [46]:
df = pd.read_excel(
    "./../../upgrade_mandi/raw-sheets-dump/DOC-20250823-WA0030..xlsx",
    sheet_name="Sheet1",
)

In [48]:
df.columns = df.columns.str.lower().str.strip()

In [50]:
locations = [
    location.locationName for location in config.domainConfigClass["Zepto"].locations
]

In [49]:
df.columns

Index(['product code', 'product name', 'uom', 'vendor name', 'nag-besa',
       'nag-bhupesh nagar', 'nag-garoba maidan', 'nag-gokulpeth',
       'nag-jaripatka', 'nag-khamla', 'nag-raghuji nagar',
       'nag-zingabai takali', 'grand total', 'rate'],
      dtype='object')

In [52]:
productDF = df[
    df.columns[[not column.startswith("nag-") for column in df.columns]]
].drop(labels=["product code", "grand total", "vendor name"], axis=1)

productDF["uom"] = productDF["uom"].apply(
    lambda uom: "#N/A" if str(uom) == "nan" else uom
)

productDF["rate"] = productDF["rate"].fillna(0)

locationDF = df[df.columns[[column.startswith("nag-") for column in df.columns]]]
locationDF.columns = [
    utils.nameExtracter(locations, column[4:]) for column in locationDF.columns
]

# locationDF.columns = [ for location in locationDF.columns]

In [ ]:
locationSepratedDF: dict[str, pd.DataFrame] = {}
for location in locations:
    locationSeries = locationDF[location]
    locationSeries.name = "invoice qty."
    locationSepratedDF[location] = pd.concat([productDF, locationSeries], axis=1)

    locationSepratedDF[location]["amount"] = (
        locationSepratedDF[location]["rate"]
        * locationSepratedDF[location]["invoice qty."]
    )

    locationSepratedDF[location].columns = [
        "article name",
        "uom",
        "rate",
        "invoice qty.",
        "amount",
    ]

    locationSepratedDF[location]["recieved qty."] = ""
    locationSepratedDF[location]["no"] = range(1, len(locationSepratedDF[location]) + 1)

    locationSepratedDF[location] = locationSepratedDF[location][
        [
            "no",
            "article name",
            "uom",
            "invoice qty.",
            "recieved qty.",
            "rate",
            "amount",
        ]
    ]

    locationSepratedDF[location] = locationSepratedDF[location].columns.str.title()

Index(['No', 'Article Name', 'Uom', 'Invoice Qty.', 'Recieved Qty.', 'Rate',
       'Amount'],
      dtype='object')